In [1]:
# import os
# import time
# import itertools

# import matplotlib.pyplot as plt
# import numpy as np
# import pandas as pd

# import torch
# from torch import nn, optim
# from torch.optim.lr_scheduler import ReduceLROnPlateau
# import torch.nn.functional as F
# from torch.utils.data import DataLoader
# from torch.utils.data.dataset import Subset
# import torchvision
# from torchvision import datasets, transforms
# from torchvision.transforms import Compose, ToTensor, Normalize
# from torchvision.datasets import MNIST

In [2]:
import torch
from torch import nn

In [3]:
from scripts.utils import *
from scripts.tensorboard import *
from scripts.dataloaders import *
from scripts.trainers import *
from scripts.models import *

In [4]:
num_classes = 11

In [16]:
# model_zoo = ['resnet18', 'resnet34', 'resnet50', 'alexnet', 'vgg', 'squeezenet', 'densenet', 'inception']

In [14]:
def test_all_models(model_zoo):
    for m in model_zoo:
        for freeze in [True, False]:
            print(m)
            model, input_size = initialize_model(m, num_classes, feature_extract=freeze)
            params = {
                'train_batch_size': 8,
                'val_batch_size': 8,
                'epochs': 1,
                'lr': 1e-3,
                'verbose': True,
                'tensorboard': False,
                'log_interval': 10,
                'log_dir': './tensorboard_logs',
                'model_name': m,
                'model': [model, input_size],
                'criterion': nn.NLLLoss(),
            #     'criterion': nn.CrossEntropyLoss(),
                'optimizer': optim.Adam,
                'early_stopping': True,
                'early_stopping_patience': 11,
                'lr_patience': 5,
                'lr_factor': 0.25,
                'min_lr': 1e-5,
            }
            
            model, optimizer, scheduler, stats = train(params, get_Arctic_data_loaders, reduce_train=0.01)
            print('__  --  __  --  __  --  __  --  __  --')
    print('Fin')

In [7]:
test_all_models(model_zoo)

resnet18
Device:  cuda
Training Results - Epoch: 1  Avg accuracy: 0.29 Avg loss: -1.09
Validation Results - Epoch: 1  Accuracy: 0.22  loss: -0.95
Final accuracy:  0.218213058419244
Confusion matrix:  tensor([[ 0,  1,  6, 12,  1,  1,  0,  0,  0,  0,  0],
        [ 0, 35,  0, 10,  5,  0,  0,  0,  0,  0,  0],
        [ 0,  6,  1,  6,  0,  0,  0,  0,  0,  2,  0],
        [ 0, 91,  3, 41, 23,  1,  0,  0,  0,  1,  1],
        [ 0, 48,  6, 56, 44,  9,  0,  0,  6,  3,  0],
        [ 0,  5, 18,  8, 23,  5,  0,  0,  0,  1,  0],
        [ 0,  2,  0,  7, 11,  0,  0,  0,  0,  0,  0],
        [ 0, 10,  0, 13,  1,  0,  0,  0,  2,  0,  0],
        [ 0,  4,  0, 27,  3,  0,  0,  0,  1,  0,  0],
        [ 0,  0,  0,  8,  3,  0,  0,  0,  0,  0,  3],
        [ 0,  3,  1,  4,  0,  0,  0,  0,  0,  0,  0]])
Precision:  tensor([0.0000, 0.1707, 0.0286, 0.2135, 0.3860, 0.3125, 0.0000, 0.0000, 0.1111,
        0.0000, 0.0000], dtype=torch.float64)
Recall:  tensor([0.0000, 0.7000, 0.0667, 0.2547, 0.2558, 0.0833, 0.

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /home/sergey/.cache/torch/checkpoints/resnet34-333f7ec4.pth
100%|██████████| 83.3M/83.3M [00:27<00:00, 3.16MB/s]


Device:  cuda
Training Results - Epoch: 1  Avg accuracy: 0.18 Avg loss: -0.76
Validation Results - Epoch: 1  Accuracy: 0.17  loss: -0.68
Final accuracy:  0.1718213058419244
Confusion matrix:  tensor([[ 0,  0,  0,  0, 19,  2,  0,  0,  0,  0,  0],
        [22,  4,  2, 13,  8,  0,  0,  1,  0,  0,  0],
        [ 0,  7,  0,  6,  2,  0,  0,  0,  0,  0,  0],
        [16, 32,  4, 21, 62,  2, 20,  0,  0,  4,  0],
        [ 1,  3,  0, 96, 71,  1,  0,  0,  0,  0,  0],
        [ 0,  0,  0, 40, 20,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0, 13,  7,  0,  0,  0,  0,  0,  0],
        [ 0,  3,  1,  3, 15,  0,  0,  4,  0,  0,  0],
        [ 0,  4,  0, 28,  1,  0,  2,  0,  0,  0,  0],
        [ 0,  0,  2,  1,  4,  3,  1,  3,  0,  0,  0],
        [ 0,  2,  1,  2,  3,  0,  0,  0,  0,  0,  0]])
Precision:  tensor([0.0000, 0.0727, 0.0000, 0.0942, 0.3349, 0.0000, 0.0000, 0.5000, 0.0000,
        0.0000, 0.0000], dtype=torch.float64)
Recall:  tensor([0.0000, 0.0800, 0.0000, 0.1304, 0.4128, 0.0000, 0.0000, 0.

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /home/sergey/.cache/torch/checkpoints/resnet50-19c8e357.pth
100%|██████████| 97.8M/97.8M [00:24<00:00, 4.14MB/s]


Device:  cuda
Training Results - Epoch: 1  Avg accuracy: 0.18 Avg loss: -1.85
Validation Results - Epoch: 1  Accuracy: 0.16  loss: -1.55
Final accuracy:  0.1718213058419244
Confusion matrix:  tensor([[  0,   0,  21,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,  15,  30,   5,   0,   0,   0,   0,   0,   0],
        [  0,   0,   4,   0,  11,   0,   0,   0,   0,   0,   0],
        [  0,   0,  42,  56,  63,   0,   0,   0,   0,   0,   0],
        [  0,   0, 119,  13,  40,   0,   0,   0,   0,   0,   0],
        [  0,   0,  43,  13,   4,   0,   0,   0,   0,   0,   0],
        [  0,   0,  17,   0,   3,   0,   0,   0,   0,   0,   0],
        [  0,   0,  19,   1,   6,   0,   0,   0,   0,   0,   0],
        [  0,   0,  15,   0,  17,   3,   0,   0,   0,   0,   0],
        [  0,   0,   6,   7,   1,   0,   0,   0,   0,   0,   0],
        [  0,   0,   4,   0,   4,   0,   0,   0,   0,   0,   0]])
Precision:  tensor([0.0000, 0.0000, 0.0131, 0.4667, 0.2597, 0.0000, 0.0000, 0.0000, 0.0000,


...

Device:  cuda
Training Results - Epoch: 1  Avg accuracy: 0.65 Avg loss: -1.73
Validation Results - Epoch: 1  Accuracy: 0.26  loss: -1.03
Final accuracy:  0.28865979381443296
Confusion matrix:  tensor([[  0,   0,   0,  19,   0,   2,   0,   0,   0,   0,   0],
        [  0,   0,   0,  41,   9,   0,   0,   0,   0,   0,   0],
        [  0,   0,   7,   1,   7,   0,   0,   0,   0,   0,   0],
        [  0,   0,  14, 105,  42,   0,   0,   0,   0,   0,   0],
        [  0,   0,  77,  48,  47,   0,   0,   0,   0,   0,   0],
        [  0,   5,   5,  18,  19,   9,   0,   0,   4,   0,   0],
        [  1,   0,   0,  16,   3,   0,   0,   0,   0,   0,   0],
        [  2,   0,   2,   6,  16,   0,   0,   0,   0,   0,   0],
        [  0,   0,   7,   4,  24,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,  13,   0,   0,   0,   0,   0,   0,   1],
        [  0,   0,   0,   3,   5,   0,   0,   0,   0,   0,   0]])
Precision:  tensor([0.0000, 0.0000, 0.0625, 0.3832, 0.2733, 0.8182, 0.0000, 0.0000, 0.0000,

Downloading: "https://download.pytorch.org/models/vgg11_bn-6002323d.pth" to /home/sergey/.cache/torch/checkpoints/vgg11_bn-6002323d.pth
100%|██████████| 507M/507M [01:18<00:00, 6.79MB/s] 


Device:  cuda
Training Results - Epoch: 1  Avg accuracy: 0.47 Avg loss: -1.27
Validation Results - Epoch: 1  Accuracy: 0.40  loss: -0.96
Final accuracy:  0.38831615120274915
Confusion matrix:  tensor([[  0,   0,   0,  18,   3,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,  40,  10,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   4,  11,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0, 138,  23,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,  84,  88,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,  45,  15,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   8,  12,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,  15,  11,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   6,  29,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,  12,   2,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   3,   5,   0,   0,   0,   0,   0,   0]])
Precision:  tensor([0.0000, 0.0000, 0.0000, 0.3700, 0.4211, 0.0000, 0.0000, 0.0000, 0.0000,

Current run is terminating due to exception: CUDA out of memory. Tried to allocate 392.00 MiB (GPU 0; 3.95 GiB total capacity; 2.27 GiB already allocated; 241.44 MiB free; 60.36 MiB cached).
Engine run is terminating due to exception: CUDA out of memory. Tried to allocate 392.00 MiB (GPU 0; 3.95 GiB total capacity; 2.27 GiB already allocated; 241.44 MiB free; 60.36 MiB cached).


RuntimeError: CUDA out of memory. Tried to allocate 392.00 MiB (GPU 0; 3.95 GiB total capacity; 2.27 GiB already allocated; 241.44 MiB free; 60.36 MiB cached)

In [17]:
test_all_models(model_zoo)

squeezenet
Device:  cuda
Training Results - Epoch: 1  Avg accuracy: 0.24 Avg loss: -2.41
Validation Results - Epoch: 1  Accuracy: 0.19  loss: -2.52
Final accuracy:  0.18384879725085912
Confusion matrix:  tensor([[  0,   0,   0,   0,  12,   0,   8,   0,   0,   1,   0],
        [  0,   4,   0,   1,   0,  11,  32,   0,   2,   0,   0],
        [  0,   5,   0,   1,   0,   9,   0,   0,   0,   0,   0],
        [  0,  12,   1,  73,   2,  27,  37,   1,   8,   0,   0],
        [  0,  15,   0,   6,   2,  10, 129,   2,   8,   0,   0],
        [  0,   0,   0,  26,   1,  14,  16,   1,   0,   0,   2],
        [  0,   0,   0,   1,   2,   0,  14,   0,   3,   0,   0],
        [  0,   0,   0,   3,   0,   0,   3,   0,  20,   0,   0],
        [  0,   4,   0,   9,   0,  15,   3,   4,   0,   0,   0],
        [  0,   6,   0,   3,   0,   2,   3,   0,   0,   0,   0],
        [  0,   3,   0,   0,   0,   3,   0,   0,   2,   0,   0]])
Precision:  tensor([0.0000, 0.0816, 0.0000, 0.5935, 0.1053, 0.1538, 0.0571, 0.00

In [ ]:
# params = {
#     'train_batch_size': 8,
#     'val_batch_size': 8,
#     'epochs': 10,
#     'lr': 1e-3,
#     'verbose': True,
#     'tensorboard': False,
#     'log_interval': 10,
#     'log_dir': './tensorboard_logs',
#     'model_name': 'ResNet18',
#     'model': [model, input_size],
#     'criterion': nn.NLLLoss(),
# #     'criterion': nn.CrossEntropyLoss(),
#     'optimizer': optim.Adam,
#     'early_stopping': True,
#     'early_stopping_patience': 11,
#     'lr_patience': 5,
#     'lr_factor': 0.25,
#     'min_lr': 1e-5,
    
# }

In [ ]:
# %%time
# model, optimizer, scheduler, stats = train(params, get_Arctic_data_loaders, reduce_train=0.01)

In [ ]:
plot_stats(stats)

In [ ]:
# %%time
# model, optimizer, scheduler, stats = continue_training(model, params, get_MNIST_data_loaders, stats, scheduler, optimizer, 
#                   new_params={
#                       'epochs': 2,
#                   })

In [ ]:
# plot_stats(stats)

In [ ]:
# model = CNN()
# model.load_state_dict(torch.load('./tensorboard_logs/MNIST/saved_models/MNIST_LeNet_lr:0.001_Adam_NLLLoss_37520.pth'))

In [ ]:
# show_batch(train_loader)